Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from dataclasses import dataclass, field



## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
    
    def copy(self) -> 'Nim':
        new_nim = Nim(num_rows=len(self._rows), k=self._k)
        new_nim._rows = list(self._rows)
        return new_nim


## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [6]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analyze(raw: Nim) -> dict:
    cooked = {"possible_moves": {}}
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analyze(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply




## (1,λ) ES


In [7]:
import random
from typing import Tuple

NUM_SETS = 5
LAMBDA = 20

@dataclass
class Individual:
    fitness: int
    genotype: Nim
    mutation_info: Tuple[int, int] = field(default_factory=lambda: (0, 0))

def fitness(state: Nim, real_state: Nim) -> int:

    xor = nim_sum(state)
    num_rows = sum(o > 0 for o in real_state.rows)
                                                 
    return (xor, num_rows)


def mutate(ind: Individual) -> Individual:
    state = list(ind.genotype.rows)

    non_empty_sets = [i for i in range(NUM_SETS) if state[i] > 0]

    pos = random.choice(non_empty_sets)
    if state[pos] != 1:
        num_to_remove = random.randint(1, state[pos]-1)
    else:
        num_to_remove = 1

    
    if len(non_empty_sets) == 1 and state[pos] != 1:
        num_to_remove = state[pos] - 1 

    state[pos] -= num_to_remove

    new_state_nim = ind.genotype.copy()
    new_state_nim.nimming(Nimply(pos, num_to_remove))

    mutated_individual = Individual(
        fitness=0,
        genotype=new_state_nim,
        mutation_info=(pos, num_to_remove),
    )

    return mutated_individual



def adaptive(state: Nim,) -> Nimply:
    offspring = []

    for _ in range(LAMBDA):
        parent = Individual(
                    fitness=0,
                    genotype=state
                )
        child = mutate(parent)
        offspring.append(child)

    for individual in offspring:
        individual.fitness = fitness(individual.genotype, state)

    best_mutated_individual = max(offspring, key=lambda ind: ind.fitness)
   
    result_nimply = Nimply(
        row=best_mutated_individual.mutation_info[0],
        num_objects=best_mutated_individual.mutation_info[1],
    )
    return result_nimply

## Oversimplified match

In [8]:
from tqdm import tqdm

num_iteration = 1000
count_0 = 0
count_1 = 0

logging.getLogger().setLevel(logging.INFO)

strategy = (adaptive, optimal)

for _ in tqdm(range(num_iteration)):
    nim = Nim(5)
    
    #logging.info(f"status: {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        #logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        #logging.info(f"status: {nim}")
        player = 1 - player
        
    if player == 0:
        count_0 += 1
    else:
        count_1 += 1

print(f"Number games won player_0: {count_0}, Percentage: {round(count_0 / num_iteration * 100)}%")
print(f"Number games won player_1: {count_1}, Percentage: {round(count_1 / num_iteration * 100)}%")


  1%|          | 10/1000 [00:00<00:11, 88.26it/s]

100%|██████████| 1000/1000 [00:09<00:00, 100.65it/s]

Number games won player_0: 425, Percentage: 42%
Number games won player_1: 575, Percentage: 57%
